In [2]:
%matplotlib inline
%env PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/ubuntu/im-photoz/Montage_v3.3/bin:/montage/bin

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import os
import shutil
import requests
import json
import bz2
import re
import subprocess
import math
from time import sleep

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import montage_wrapper as mw
from astropy.io import fits
from astropy import wcs

import warnings
warnings.filterwarnings("ignore")

env: PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/ubuntu/im-photoz/Montage_v3.3/bin:/montage/bin


In [3]:
# SQL to get data
def sample_one_sq_deg(ra, dec, d=1, n=500, dr="dr12", timeout=60):
    
    url = "http://skyserver.sdss.org/SkyserverWS/{}/SearchTools/SqlSearch".format(dr)
    payload = {
        "format": "json",
        "cmd": """
            SELECT TOP {0} spec.specObjID, phot.objID,
                spec.ra, spec.dec,
                spec.class,
                spec.z, spec.zErr,
                phot.rerun, phot.run, phot.camcol, phot.field,
                phot.dered_u, phot.dered_g, phot.dered_r, phot.dered_i, phot.dered_z,
                phot.psfMag_u, phot.psfMag_g, phot.psfMag_r, phot.psfMag_i, phot.psfMag_z,
                phot.extinction_u, phot.extinction_g, phot.extinction_r, phot.extinction_i, phot.extinction_z
            FROM SpecObjAll AS spec
            JOIN PhotoObjAll AS phot
            ON spec.specObjID = phot.specObjID
            WHERE
                phot.clean = 1
                AND spec.zWarning = 0
                AND spec.ra >= {1}
                AND spec.ra < {2}
                AND spec.dec >= {3}
                AND spec.dec < {4}
                AND phot.dered_r > 0 and phot.dered_r < 40
                AND phot.expRad_r < 30
                AND phot.deVRad_r < 30
                AND spec.zErr < 0.1
                AND spec.z < 2
                AND spec.class = 'galaxy'
            ORDER BY NEWID()
        """.format(n, ra, ra + d, dec, dec + d).strip()
    }
    
    try:
        resp = requests.post(url, params=payload, timeout=timeout)
    except requests.exceptions.RequestException as e:
        print(e)
        return None
    
    data = resp.json()[0]['Rows']
    
    df = pd.DataFrame(data)
    
    df[["specObjID", "objID"]] = df[["specObjID", "objID"]].astype("object")

    return df

In [4]:
# Download images
def fetch_fits(df, dirname="temp"):

    bands = [c for c in 'ugriz']

    if not os.path.exists(dirname):
        os.makedirs(dirname)

    for i, r in df.iterrows():

        url = "http://data.sdss3.org/sas/dr12/boss/photoObj/frames/{0}/{1}/{2}/".format(
            r["rerun"], r["run"], r["camcol"], r["field"])

        print("Downloading rerun: {}, run: {}, camcol: {}, field:{}".format(
            r["rerun"], r["run"], r["camcol"], r["field"]))
    
        for band in bands:

            filename = "frame-{4}-{1:06d}-{2}-{3:04d}.fits".format(
                r["rerun"], r["run"], r["camcol"], r["field"], band)
            filepath = os.path.join(dirname, filename)
            
            if os.path.exists(filepath):
                continue

            for _ in range(10):
                try:
                    resp = requests.get(url + filename + ".bz2")
                except:
                    sleep(1)
                    continue
                
                if resp.status_code == 200:
                    with open(filepath, "wb") as f:
                        img = bz2.decompress(resp.content)
                        f.write(img)
                    break
                else:
                    sleep(1)
                    continue

            if not os.path.exists(filepath):
                raise Exception

In [5]:
def get_ref_list(df):

    ref_images = []
    
    for row in df.iterrows():
        r = row[1]
        filename = "frame-r-{1:06d}-{2}-{3:04d}.fits".format(r["rerun"], r["run"], r["camcol"], r["field"])
        ref_images.append(filename)

    return ref_images

### Montage

In [6]:
# Montage: align images
def align_images(images, frame_dir="temp", registered_dir="temp/registered/"):

    if not os.path.exists(registered_dir):
        os.makedirs(registered_dir)
    
    for image in images:

        registered_path = [
            os.path.join(registered_dir, image.replace("frame-r-", "registered-{}-").format(b))
            for b in "ugriz"
            ]
        
        if all([os.path.exists(r) for r in registered_path]):
            print("Skipping {}...".format(image))
            continue
        else:
            print("Processing {}...".format(image))
    
        frame_path = [
            os.path.join(frame_dir, image.replace("frame-r-", "frame-{}-").format(b))
            for b in "ugriz"
            ]

        header = os.path.join(
            registered_dir,
            image.replace("frame", "header").replace(".fits", ".hdr")
            )

        mw.commands.mGetHdr(os.path.join(frame_dir, image), header)
        mw.reproject(
            frame_path, registered_path,
            header=header, silent_cleanup=True, common=True
            )
    
    return None

In [7]:
# Convert ra and dec values to pixel positions
def convert_catalog_to_pixels(df, dirname="temp/registered/"):

    if not os.path.exists(dirname):
        os.makedirs(dirname)

    pixels = []
    fits_list = []

    for i, r in df.iterrows():

        fits_file = "registered-r-{1:06d}-{2}-{3:04d}.fits".format(
            r["rerun"], r["run"], r["camcol"], r["field"])
        fits_path = os.path.join(dirname, fits_file)
            
        hdulist = fits.open(fits_path)

        w = wcs.WCS(hdulist[0].header, relax=False)
        
        px, py = w.all_world2pix(r["ra"], r["dec"], 1)

        fits_list.append(fits_file)
        pixels.append((i, px, py))

    for i, fits_file in enumerate(fits_list):
        ix, px, py = pixels[i]
        pixel_list = fits_file.replace(".fits", ".list")
        pixel_path = os.path.join(dirname, pixel_list)
        with open(pixel_path, "a") as fout:
            fout.write("{} {} {}\n".format(ix, px, py))
            
    obj_pixels = pd.DataFrame(pixels, columns=['id', 'object_x_coord', 'object_y_coord']).drop('id', axis=1)
    df_new = df.copy()
    df_new['object_x_coord'] = obj_pixels['object_x_coord']
    df_new['object_y_coord'] = obj_pixels['object_y_coord']

    return df_new

### Sextractor

In [8]:
%%writefile default.param
XMIN_IMAGE               Minimum x-coordinate among detected pixels                [pixel]
YMIN_IMAGE               Minimum y-coordinate among detected pixels                [pixel]
XMAX_IMAGE               Maximum x-coordinate among detected pixels                [pixel]
YMAX_IMAGE               Maximum y-coordinate among detected pixels                [pixel]
VECTOR_ASSOC(1)          #ASSOCiated parameter vector

Overwriting default.param


In [9]:
%%writefile default.sex
#-------------------------------- Catalog ------------------------------------
 
CATALOG_NAME     test.cat       # name of the output catalog
CATALOG_TYPE     ASCII_HEAD     # NONE,ASCII,ASCII_HEAD, ASCII_SKYCAT,
                                # ASCII_VOTABLE, FITS_1.0 or FITS_LDAC
PARAMETERS_NAME  default.param  # name of the file containing catalog contents
 
#------------------------------- Extraction ----------------------------------
 
DETECT_TYPE      CCD            # CCD (linear) or PHOTO (with gamma correction)
DETECT_MINAREA   3              # min. # of pixels above threshold
DETECT_THRESH    1.5            # <sigmas> or <threshold>,<ZP> in mag.arcsec-2
ANALYSIS_THRESH  1.5            # <sigmas> or <threshold>,<ZP> in mag.arcsec-2
 
FILTER           Y              # apply filter for detection (Y or N)?
FILTER_NAME      default.conv   # name of the file containing the filter
 
DEBLEND_NTHRESH  32             # Number of deblending sub-thresholds
DEBLEND_MINCONT  0.005          # Minimum contrast parameter for deblending
 
CLEAN            Y              # Clean spurious detections? (Y or N)?
CLEAN_PARAM      1.0            # Cleaning efficiency
 
MASK_TYPE        CORRECT        # type of detection MASKing: can be one of
                                # NONE, BLANK or CORRECT

#------------------------------ Photometry -----------------------------------
 
PHOT_APERTURES   5              # MAG_APER aperture diameter(s) in pixels
PHOT_AUTOPARAMS  2.5, 3.5       # MAG_AUTO parameters: <Kron_fact>,<min_radius>
PHOT_PETROPARAMS 2.0, 3.5       # MAG_PETRO parameters: <Petrosian_fact>,
                                # <min_radius>

SATUR_LEVEL      50000.0        # level (in ADUs) at which arises saturation
SATUR_KEY        SATURATE       # keyword for saturation level (in ADUs)
 
MAG_ZEROPOINT    0.0            # magnitude zero-point
MAG_GAMMA        4.0            # gamma of emulsion (for photographic scans)
GAIN             0.0            # detector gain in e-/ADU
GAIN_KEY         GAIN           # keyword for detector gain in e-/ADU
PIXEL_SCALE      1.0            # size of pixel in arcsec (0=use FITS WCS info)
 
#------------------------- Star/Galaxy Separation ----------------------------
 
SEEING_FWHM      1.2            # stellar FWHM in arcsec
STARNNW_NAME     default.nnw    # Neural-Network_Weight table filename
 
#------------------------------ Background -----------------------------------
 
BACK_SIZE        64             # Background mesh: <size> or <width>,<height>
BACK_FILTERSIZE  3              # Background filter: <size> or <width>,<height>
 
BACKPHOTO_TYPE   GLOBAL         # can be GLOBAL or LOCAL
 
#------------------------------ Check Image ----------------------------------
 
CHECKIMAGE_TYPE  SEGMENTATION   # can be NONE, BACKGROUND, BACKGROUND_RMS,
                                # MINIBACKGROUND, MINIBACK_RMS, -BACKGROUND,
                                # FILTERED, OBJECTS, -OBJECTS, SEGMENTATION,
                                # or APERTURES
CHECKIMAGE_NAME  check.fits     # Filename for the check-image
 
#--------------------- Memory (change with caution!) -------------------------
 
MEMORY_OBJSTACK  3000           # number of objects in stack
MEMORY_PIXSTACK  300000         # number of pixels in stack
MEMORY_BUFSIZE   1024           # number of lines in buffer
 
#----------------------------- Miscellaneous ---------------------------------
 
VERBOSE_TYPE     NORMAL         # can be QUIET, NORMAL or FULL
HEADER_SUFFIX    .head          # Filename extension for additional headers
WRITE_XML        N              # Write XML file (Y/N)?
XML_NAME         sex.xml        # Filename for XML output

#----------------------------- ASSOC parameters ---------------------------------

ASSOC_NAME       sky.list       # name of the ASCII file to ASSOCiate, the expected pixel 
                                # coordinates list given as [id, xpos, ypos]
ASSOC_DATA       1              # columns of the data to replicate (0=all), replicate id
                                # of the object in the SExtractor output file
ASSOC_PARAMS     2,3            # columns of xpos,ypos[,mag] in the expected pixel
                                # coordinates list
ASSOC_RADIUS     2.0            # cross-matching radius (pixels)
ASSOC_TYPE       NEAREST        # ASSOCiation method: FIRST, NEAREST, MEAN,
                                # MAG_MEAN, SUM, MAG_SUM, MIN or MAX
ASSOCSELEC_TYPE  MATCHED        # ASSOC selection type: ALL, MATCHED or -MATCHED

Overwriting default.sex


In [10]:
def run_sex(filename, dirname="temp/registered/"): # Run sextractor on one file
    
    fpath = os.path.join(dirname, filename)
        
    list_file = filename.replace(".fits", ".list")
    list_path = os.path.join(dirname, list_file)

    config_file = filename.replace(".fits", ".sex")

    with open("default.sex", "r") as default:
        with open(config_file, "w") as temp:
            for line in default:
                line = re.sub(
                    r"^ASSOC_NAME\s+sky.list",
                    "ASSOC_NAME       {}".format(list_file),
                    line
                )
                temp.write(line)
    
    shutil.copy(list_path, os.getcwd())
    
    subprocess.call(["sex", "-c", config_file, fpath])

    os.remove(config_file)
    os.remove(os.path.join(os.getcwd(), list_file))

In [11]:
def nanomaggie_to_luptitude(array, band):
    '''
    Converts nanomaggies (flux) to luptitudes (magnitude).

    http://www.sdss.org/dr12/algorithms/magnitudes/#asinh
    http://arxiv.org/abs/astro-ph/9903081
    '''
    b = {
        'u': 1.4e-10,
        'g': 0.9e-10,
        'r': 1.2e-10,
        'i': 1.8e-10,
        'z': 7.4e-10
    }
    nanomaggie = array * 1.0e-9 # fluxes are in nanomaggies

    luptitude = -2.5 / np.log(10) * (np.arcsinh((nanomaggie / (2 * b[band]))) + np.log(b[band]))
    
    return luptitude

In [12]:
def get_pixels_position(df, registered_all):
    # Construct the new DataFrame with pixels of the stellar objects.
    cat = pd.DataFrame()
    pixels = pd.DataFrame()

    remaining = len(registered_all)
    print ('Total number of images to pass = {num}'.format(num=remaining))
    for f in registered_all:
        run_sex(f)
    
        # Get cat
        try:
            assoc = pd.read_csv(
                "test.cat",
                skiprows=5,
                sep="\s+",
                names=["xmin", "ymin", "xmax", "ymax", "match"]
            )
            assoc["file"] = f
            cat = cat.append(assoc)
        except:
            pass
    
        # Get pixels, use r band as a reference
        mask = fits.open('check.fits')  
        m = mask[0].data
        image = fits.open('temp/registered/'+f)
        img = image[0].data
    
        # Get flux from all bands (ugriz)
        flux = {}
        flux_err = {}
        bands = ['u', 'g', 'r', 'i', 'z']
        for b in bands:
            path = f.replace("-r-", "-{}-".format(b))
            image = fits.open('temp/registered/'+path)
            flux[b] = image[0].data
        f_u, f_g, f_r, f_i, f_z = flux.values()
    
        # Leave pixels that belong to objects only
        img[m == 0] = 99 # Background
        for objnum in np.unique(m):
            if objnum != 0:

                px, py = np.where(m == objnum)
                    
                u = f_u[m == objnum]
                g = f_g[m == objnum]
                r = f_r[m == objnum]
                i = f_i[m == objnum]
                z = f_z[m == objnum]

                mat = assoc.loc[objnum-1]['match']
                match = np.full(len(px), mat)
                pix = pd.DataFrame({'match': match, 'pixel_x_coord': px, 'pixel_y_coord': py,
                                    'flux_u': u, 'flux_g': g, 'flux_r':r, 'flux_i':i, 'flux_z':z,
                                    'flux_err_u': u, 'flux_err_g': g, 'flux_err_r':r, 'flux_err_i':i, 'flux_err_z':z})
                pixels = pixels.append(pix)
            
        remaining -= 1
        print ('Remaining number of images = {num}'.format(num=remaining))
    
    print ('Matching starts.')
    if len(cat) > 0:
        cat["class"] = df.ix[cat["match"], "class"].values
        cat["objID"] = df.ix[cat["match"], "objID"].values
    if len(pixels) > 0:
        pixels["objID"] = df.ix[pixels["match"], "objID"].values
        pixels["class"] = df.ix[pixels["match"], "class"].values 
        pixels['object_x_coord'] = df.ix[pixels["match"], 'object_x_coord'].values.astype(float)
        pixels['object_y_coord'] = df.ix[pixels["match"], 'object_y_coord'].values.astype(float)
        pixels["z"] = df.ix[pixels["match"], "z"].values
        pixels["zErr"] = df.ix[pixels["match"], "zErr"].values
        pixels["extinction_u"] = df.ix[pixels["match"],"extinction_u"].values
        pixels["extinction_g"] = df.ix[pixels["match"],"extinction_g"].values
        pixels["extinction_r"] = df.ix[pixels["match"],"extinction_r"].values
        pixels["extinction_i"] = df.ix[pixels["match"],"extinction_i"].values
        pixels["extinction_z"] = df.ix[pixels["match"],"extinction_z"].values
        if not "pixelID" in pixels.columns:
            pixels.insert(loc=0, value = np.arange(0, pixels['pixel_x_coord'].count()), column = "pixelID")
    
    # Calculate distance
    if len(pixels) > 0:
        x_pixels = np.array(pixels.pixel_x_coord)
        x_obj_pixels = np.array(pixels.object_x_coord)
        y_pixels = np.array(pixels.pixel_y_coord)
        y_obj_pixels = np.array(pixels.object_y_coord)
        pixels["distance_to_obj"] = ((x_pixels-x_obj_pixels)**2+(y_pixels-y_obj_pixels)**2)**0.5
    
    # Change fluxes from nanomaggie to luptitude
    pixels["lup_u"] = nanomaggie_to_luptitude(np.array(pixels.flux_u), 'u')
    pixels["lup_g"] = nanomaggie_to_luptitude(np.array(pixels.flux_g), 'g')
    pixels["lup_r"] = nanomaggie_to_luptitude(np.array(pixels.flux_r), 'r')
    pixels["lup_i"] = nanomaggie_to_luptitude(np.array(pixels.flux_i), 'i')
    pixels["lup_z"] = nanomaggie_to_luptitude(np.array(pixels.flux_z), 'z')
    
    print ('The job is finished.')
    
    return cat, pixels

In [27]:
def main(ra, dec):
    df = sample_one_sq_deg(ra, dec, d=1, n=500)
    print(len(df))
    fetch_fits(df)
    ref_images = get_ref_list(df)
    align_images(ref_images)
    objects = convert_catalog_to_pixels(df)
    registered_all = [f.replace("frame-", "registered-") for f in ref_images]
    cat, pixels = get_pixels_position(objects, registered_all)
    pixels.to_csv("data/pixels/{ra},{dec}-pixels.csv".format(ra=ra, dec=dec), index=False)
    # objects.to_csv("data/objects/{ra},{dec}-objects.csv".format(ra=ra, dec=dec), index=False)
    shutil.rmtree("temp/")
    return None